<a href="https://colab.research.google.com/github/ShehabMMohamed/TensorFlow-Tutorials/blob/main/Recurrent_Neural_Network_For_Movie_Reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![](https://drive.google.com/uc?export=view&id=1pf7Is3PTuW9iu-VUIs-FhlFBXrsmydhf)

**Recurrent Neural Networks**

Type of Neural Networks that are good at modelling sequence data. 

RNNs read the data sequentially in steps before making a prediction or classification. Think of it the same way you say out loud the alphabets, it is easier to remember based on sequence, but if you start saying the alphatets from the letter 'I' it becomes slightly challenging.


![](https://drive.google.com/uc?export=view&id=1Y3U7KJYm5BYONCoEpzJAayTJIYi_YIdn)



```
# Basic Idea Behind an RNN.
rnn = RNN()
ff = FeedForwardNN()
hidden_state = [0.0, 0.0, 0.0, 0.0]

for word in input:
  output, hidden_state = rnn(word, hidden_state)

prediction = ff(output)
```


**Variations of Recurrent Neural Networks which are capable of learning long term dependencies using mechanism called gates**
- Long Short Term Memory (LSTM)
- Gated Recurrent Unit (GRU)

Gates are a way to optionally let information through. 

Highly recommended resource for understanding about LSTMs.
https://colah.github.io/posts/2015-08-Understanding-LSTMs/



### Different Sequence Problems used in Recurrent Neural Networks

![](https://drive.google.com/uc?export=view&id=1lDjgihc9KOxpTJ2xUFRYlxL3q8IRXuRc)

In [1]:
%tensorflow_version 2.x 
from keras.datasets import imdb             # one of many available datasets to try out. https://www.tensorflow.org/datasets/catalog/imdb_reviews
from keras.preprocessing import sequence    # We need to perform sequence padding to the text to guarantee same input length
import tensorflow as tf                     # Deep Learning Library
import numpy as np                          # For Fast Matrix Multiplication

VOCAB_SIZE = 88584    # Number of Unique Words
MAXLEN = 250        # Maximum length of a sentence

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words = VOCAB_SIZE)

17465344/17464789 [==============================] - 0s 0us/step


In [2]:
print(train_data.shape)
type(train_data)

(25000,)


numpy.ndarray

In [3]:
# examples of train_data and train_label
def show_train_examples(n = 5, flag = True):
  for i in range(n):
    id = i
    print("Shape of Input Data [{0}]: {1}".format(id, len(train_data[id])))
    if flag:
      print("Train: ", train_data[id])
      print("Label: ", train_labels[id])

# Notice the different lengths of the inputs.
# The Movie reviews are not of equal length, we need to pad them to a fixed-length vector.
show_train_examples(2, True)

Shape of Input Data [0]: 218
Train:  [1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 22665, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 21631, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 19193, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 10311, 8, 4, 107, 117, 5952, 15, 256, 4, 31050, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 12118, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]

Padding the text to MAXLEN=250

In [4]:
train_data = sequence.pad_sequences(train_data, MAXLEN) 
test_data = sequence.pad_sequences(test_data, MAXLEN)

# Now all reviews are padded to 250 length.
show_train_examples(10, False)

Shape of Input Data [0]: 250
Shape of Input Data [1]: 250
Shape of Input Data [2]: 250
Shape of Input Data [3]: 250
Shape of Input Data [4]: 250
Shape of Input Data [5]: 250
Shape of Input Data [6]: 250
Shape of Input Data [7]: 250
Shape of Input Data [8]: 250
Shape of Input Data [9]: 250


**What is Word Embedding?**

![](https://drive.google.com/uc?export=view&id=1y9YsZh5bnxDUH31uy9Y-Hg5Pqt9WnF0_)

If we have the embedding size of 512, then we have a weighted matrix of (VOCAB_SIZE, EMBEDDING_SIZE) where each row is a word with a vector of 512 numerical values. 

After training this embedding layer, we can use it as a lookup table! Input a word in the embedding layer and it will output for us its embeddings.

Architecture of our Recurrent Neural Network

- Input Layer: Embedd the words with vectors of dimensionality 32.
- LSTM Layer: Input the embeddings to an LSTM layer with 32 neurons.
- Output Layer: output with a single neuron with output value from 0.0 to 1.0 (Sigmoid function)

Remember
Embedding layer is of shape (Number of Words x Embedding Dimensions)


In [5]:
model = tf.keras.Sequential([ 
                             tf.keras.layers.Embedding(VOCAB_SIZE, 32),     # Input Layer (1)
                             tf.keras.layers.LSTM(32),                      # Hidden Layer
                             tf.keras.layers.Dense(1, activation='sigmoid') # Output Layer
])


View the architecture of our model using summary() function.

In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 32)          2834688   
_________________________________________________________________
lstm (LSTM)                  (None, 32)                8320      
_________________________________________________________________
dense (Dense)                (None, 1)                 33        
Total params: 2,843,041
Trainable params: 2,843,041
Non-trainable params: 0
_________________________________________________________________


Hyperparameter Tuning


What if you train with a different optimizer like Adam?
What if you train with more epochs? Is there a stopping condition during training? Observing the validation loss?
What if you change the validation split to 15%?
Change the RNN architecture?

In [7]:

model.compile(loss="binary_crossentropy", optimizer="rmsprop", metrics=["acc"])

history = model.fit(train_data, train_labels, epochs=3, validation_split=0.2)

Epoch 1/3
625/625 [==============================] - 15s 24ms/step - loss: 0.4189 - acc: 0.8115 - val_loss: 0.3218 - val_acc: 0.8728
Epoch 2/3
625/625 [==============================] - 14s 23ms/step - loss: 0.2383 - acc: 0.9073 - val_loss: 0.3659 - val_acc: 0.8678
Epoch 3/3
625/625 [==============================] - 14s 23ms/step - loss: 0.1849 - acc: 0.9323 - val_loss: 0.2775 - val_acc: 0.8980


Test the Accuracy with the test_data.

In [8]:
results = model.evaluate(test_data, test_labels)

782/782 [==============================] - 4s 6ms/step - loss: 0.3213 - acc: 0.8780


Encoding Text function takes an input text and transforms it to an array of numerical values.

Decoding Text function takes the array of numerical values and transforms it to an input text.

In [10]:
# We need this dictionary to encode the text.
# This dictionary has the mapping from a word to an integer.
word_index = imdb.get_word_index() 

def encode_text(text):
  tokens = tf.keras.preprocessing.text.text_to_word_sequence(text)
  # for word in tokens:
  #   if word in word_index:
  #     tokens.append(word_index[word])
  #   else:
  #     tokens.append(0)
  tokens = [word_index[word] if word in word_index else 0 for word in tokens]   # List Comprehension
  return sequence.pad_sequences([tokens], MAXLEN)[0]

text = "that movie was just amazing, so amazing"
encoded = encode_text(text)
print("Input Text: ", text)
print("Encoded Text: ", encoded)

Input Text:  that movie was just amazing, so amazing
Encoded Text:  [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0

In [11]:
# We need this dictionary to decode the integers.
# This dictionary has the mapping from an integer to a corresponding word.
reverse_word_index = {value: key for (key, value) in word_index.items()}    # Dictionary Comprehension

def decode_integers(integers):
  PAD = 0
  text = ""
  for num in integers:
    if num != PAD:
      text += reverse_word_index[num] + " "
  return text[:-1]
  
print("Encoded Text: " , text)
print("Decoded Text: ", decode_integers(encoded))

Encoded Text:  that movie was just amazing, so amazing
Decoded Text:  that movie was just amazing so amazing


After training our network, we are going to test the model with some input text.

In [12]:
def predict(text):
  encoded_text = encode_text(text)
  pred = np.zeros((1, MAXLEN))    # Batch of Inputs[[review]]
  pred[0] = encoded_text          # pred[0] = [review]
  result = model.predict(pred)
  print(result[0])

positive_review = "That movie was so awesome! I really loved it and would watch it again because it was amazingly great"
predict(positive_review)

negative_review = "That movie sucked. I hated it and wouldn't watch it again. Was one of the worst things I've ever watched"
predict(negative_review)

[0.77847785]
[0.34788582]


In [14]:
def predict_and_assign_sentiment(text):
  encoded_text = encode_text(text)
  pred = np.zeros((1, MAXLEN))
  pred[0] = encoded_text
  result = model.predict(pred)
  review = "Positive" if result > 0.5 else "Negative"
  print("Input Text: ", text)
  print("It is a {0} review!".format(review))

predict_and_assign_sentiment("I am not sure about this movie, it was weird for me. I did not like it.")
predict_and_assign_sentiment("I will definitely want to see it again!")
predict_and_assign_sentiment("This movie was good, lol joking!")

Input Text:  I am not sure about this movie, it was weird for me. I did not like it.
It is a Negative review!
Input Text:  I will definitely want to see it again!
It is a Positive review!
Input Text:  This movie was good, lol joking!
It is a Negative review!
